In [3]:
import pandas as pd 

In [217]:
ann_data_small_bbox = pd.read_csv("/fast_data_e2e_1/cxr/qxr_ln_data/cxr_data/filt_csvs/test_nodule_image_positive_with_big_bbox_removed.csv")
tag_ex_akhila = pd.read_csv('/fast_data_e2e_1/cxr/qxr_ln_data/cxr_data/filt_csvs/bbox_list_big_bbox_tags_test_data.csv')
tag_ex_old = pd.read_csv('/models_common_e2e/cxr_data/testing/testing_master_4_4_24_tag_corrected_tb_4_tags.csv')
master_test_csv = pd.read_csv('/fast_data_e2e11/qxr_ln_trainings/testing_shreshtha/master_test_csv_13-08-24.csv')

In [218]:
for col in tag_ex_akhila.columns:
    # Check if the column is of type float64
    if tag_ex_akhila[col].dtype == 'float64':
        # Check if the column can be safely converted back to int without data loss
        if (tag_ex_akhila[col] % 1 == 0).all():
            tag_ex_akhila[col] = tag_ex_akhila[col].astype('int64')


In [219]:
## merging docs annots with tag ex and removing big bbox 
tag_ex_akhila.loc[tag_ex_akhila['big_bbox'] == 0, 'nodule'] = 1
tag_ex_akhila.loc[tag_ex_akhila['big_bbox'] == 1, 'nodule'] = 0

In [220]:
tag_ex_akhila.big_bbox.value_counts()

big_bbox
-100    227884
 0        6373
 1        2833
Name: count, dtype: int64

In [221]:
big_bbox_filenames = tag_ex_akhila[tag_ex_akhila['big_bbox'] == 1 ]['filename'].to_list()

In [222]:
columns_to_check = ['fibronodule',  'fibrolesion', 'miliarypattern', 'infectious_nodule', 'tubr_nodule']
tag_ex_akhila.loc[tag_ex_akhila[columns_to_check].eq(1).any(axis=1), 'nodule'] = 0

In [223]:
tag_ex_akhila_ = tag_ex_akhila[['filename' , 'nodule',  'nipple_shadow','nipple_marker' , 'fibronodule', 'fibrolesion', 'miliarypattern', 'infectious_nodule', 'tubr_nodule']]

In [224]:
tag_ex_akhila_nodulefilenames = tag_ex_akhila[tag_ex_akhila['nodule'] == 1 ]['filename'].to_list()

In [225]:
tag_ex_old_ = tag_ex_old[['filename','nodule', 'calcification',
       'granuloma', 'mass', 'ribfracture', 'consolidation', 'fibrosis']]

In [226]:
for col in tag_ex_old_.columns:
    # Check if the column is of type float64
    if tag_ex_old_[col].dtype == 'float64':
        # Check if the column can be safely converted back to int without data loss
        if (tag_ex_old_[col] % 1 == 0).all():
            tag_ex_old_[col] = tag_ex_old_[col].astype('int64')

/tmp/ipykernel_46925/4024944712.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tag_ex_old_[col] = tag_ex_old_[col].astype('int64')


In [227]:
## removing big bbox data ( from annotations we have done ) from old tag ex on all reports nodule positives 
tag_ex_old_.loc[tag_ex_old_['filename'].isin(big_bbox_filenames), 'nodule'] = 0


In [228]:
## taking a union of old tax ex results and our csv with nodules from both tag ex and annots from doc with 'fibronodule', 'fibrolesion', 'miliarypattern', 'infectious_nodule', 'tubr_nodule' removed 
tag_ex_old_.loc[tag_ex_old_['filename'].isin(tag_ex_akhila_nodulefilenames), 'nodule'] = 1

In [229]:
tag_ex_old_.nodule.value_counts()

nodule
 0      293825
-100     20940
 1        7462
Name: count, dtype: int64

In [230]:
columns_to_merge = ['filename', 'nipple_shadow' , 'nipple_marker', 'fibronodule', 'fibrolesion', 'miliarypattern', 'infectious_nodule', 'tubr_nodule']
merged_df = tag_ex_old_.merge(tag_ex_akhila_[columns_to_merge], on='filename', how='left')
for col in columns_to_merge[1:]:  # Skip 'filename'
    merged_df[col] = merged_df[col].fillna(-100)

In [231]:
df_tag_ex_old_updated =  merged_df.dropna().reset_index(drop =True)

In [232]:
df_tag_ex_old_updated.nodule.value_counts()

nodule
 0      293825
-100     20940
 1        7462
Name: count, dtype: int64

In [233]:
for col in df_tag_ex_old_updated.columns:
    # Check if the column is of type float64
    if df_tag_ex_old_updated[col].dtype == 'float64':
        # Check if the column can be safely converted back to int without data loss
        if (df_tag_ex_old_updated[col] % 1 == 0).all():
            df_tag_ex_old_updated[col] = df_tag_ex_old_updated[col].astype('int64')

In [245]:
len(df_tag_ex_old_updated.columns) 

16

In [261]:
master_test_csv.columns

Index(['filename', 'src', 'nodule_gt', 'nipple_shadow', 'calcification',
       'granuloma', 'mass', 'ribfracture', 'consolidation', 'fibrosis',
       'ct recommended', 'solitary', 'artifact', 'subtlety', 'nipple_marker',
       'fibronodule', 'fibrolesion', 'miliarypattern', 'infectious_nodule',
       'tubr_nodule', 'LN', 'LNMS', 'bbox', 'png_path'],
      dtype='object')

'ct recommended', 'solitary', 'artifact', 'subtlety' , 'png_path'

In [236]:
## creating a LN tag
columns_to_check = ['nipple_shadow', 'granuloma']
df_tag_ex_old_updated['LN'] = df_tag_ex_old_updated['nodule']
df_tag_ex_old_updated.loc[df_tag_ex_old_updated[columns_to_check].eq(1).any(axis=1), 'LN'] = 1

In [248]:
df_tag_ex_old_updated['LNMS'] = -100
df_tag_ex_old_updated['bbox'] = [[] for _ in range(len(df_tag_ex_old_updated))]
df_tag_ex_old_updated['src'] = 'internal testset'

In [253]:
master_test_csv_internal = master_test_csv[master_test_csv['src'] == 'internal testset']

In [255]:
master_test_csv_internal

,filename,src,nodule_gt,nipple_shadow,calcification,granuloma,mass,ribfracture,consolidation,fibrosis,...,nipple_marker,fibronodule,fibrolesion,miliarypattern,infectious_nodule,tubr_nodule,LN,LNMS,bbox,png_path
218997,ca.50kstudies.unit6.a6b19ade890324758bc58aa5ad...,internal testset,0,-100,0,0,0,0,0,0,...,-100,-100,-100,0,-100,-100,0,-100,[],/models_common_e2e/cxr_data/testing/images/ca....
218998,naw.1.3.51.0.7.123293555.65006.58701.36014.869...,internal testset,0,-100,0,0,0,0,1,0,...,-100,-100,-100,0,-100,-100,0,-100,[],/models_common_e2e/cxr_data/testing/images/naw...
218999,dmims.1.2.392.200036.9107.307.15120.20190123.1...,internal testset,0,-100,0,0,0,0,0,0,...,-100,-100,-100,0,-100,-100,0,-100,[],/models_common_e2e/cxr_data/testing/images/dmi...
219000,krsnaa.1.2.392.200036.9125.4.0.487578536.53798...,internal testset,0,-100,0,0,0,0,0,0,...,-100,-100,-100,0,-100,-100,0,-100,[],/models_common_e2e/cxr_data/testing/images/krs...
219001,krsnaa.1.2.392.200036.9125.9.0.253290553.41978...,internal testset,0,-100,0,0,0,0,1,1,...,-100,-100,-100,0,-100,-100,0,-100,[],/models_common_e2e/cxr_data/testing/images/krs...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
533345,krsnaa.1.2.392.200036.9125.4.0.487523815.25243...,internal testset,0,-100,0,0,0,0,0,0,...,0,0,0,0,0,0,1,-100,[],/models_common_e2e/cxr_data/testing/images/krs...
533346,naw.1.2.840.113564.0.224.76.54.83.182.20190117...,internal testset,1,-100,0,0,0,0,0,0,...,0,0,0,0,0,0,1,-100,[],/models_common_e2e/cxr_data/testing/images/naw...
533347,naw.1.2.392.200036.9125.2.249636124116150.6488...,internal testset,0,-100,0,0,0,0,1,0,...,0,0,0,0,0,0,1,-100,[],/models_common_e2e/cxr_data/testing/images/naw...
533348,krsnaa.1.2.392.200036.9125.9.0.253496398.14766...,internal testset,0,-100,0,0,0,0,0,0,...,0,0,0,0,0,0,1,-100,[],/models_common_e2e/cxr_data/testing/images/krs...


In [256]:
master_test_csv_internal_ = master_test_csv_internal[['filename', 'ct recommended', 'solitary', 'artifact', 'subtlety' , 'png_path']]

In [258]:
master_test_csv_internal_ = master_test_csv_internal_.merge(df_tag_ex_old_updated, on ="filename", how ='left')

In [259]:
len(master_test_csv_internal_.columns)

24

In [263]:
order = ['filename', 'src', 'nodule', 'nipple_shadow', 'calcification',
       'granuloma', 'mass', 'ribfracture', 'consolidation', 'fibrosis',
       'ct recommended', 'solitary', 'artifact', 'subtlety', 'nipple_marker',
       'fibronodule', 'fibrolesion', 'miliarypattern', 'infectious_nodule',
       'tubr_nodule', 'LN', 'LNMS', 'bbox', 'png_path']
master_test_csv_internal_ = master_test_csv_internal_[order]

In [266]:
master_test_csv_internal_.dropna(inplace=True)


In [267]:
master_test_csv_internal_.LN.value_counts()

LN
 0      285925
-100     20939
 1        7489
Name: count, dtype: int64

In [268]:
master_test_csv_internal_.nodule.value_counts()

nodule
 0      286020
-100     20940
 1        7393
Name: count, dtype: int64

In [279]:
master_test_csv_rest = master_test_csv[master_test_csv['src'] != 'internal testset']
master_test_csv_rest.rename(columns = {'nodule_gt' : 'nodule' }, inplace = True)

/tmp/ipykernel_46925/2780678262.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  master_test_csv_rest.rename(columns = {'nodule_gt' : 'nodule' }, inplace = True)


In [280]:
combined_df = pd.concat([master_test_csv_internal_, master_test_csv_rest], axis=0, ignore_index=True)


In [283]:
combined_df.dropna(inplace =True)

In [284]:
combined_df.nodule.value_counts()

nodule
 0      493801
-100     20940
 1       18609
Name: count, dtype: int64

In [286]:
combined_df.LN.value_counts()

LN
 0      492688
-100     20939
 1       19723
Name: count, dtype: int64

In [290]:
combined_df.to_csv("/fast_data_e2e_1/cxr/qxr_ln_data/LN_test/combined_test_csv_updated_internal_test_02-09-24.csv", index = False)

### include FiT cancer data 

In [1]:
import pandas as pd 

In [2]:
combined_test = pd.read_csv("/fast_data_e2e_1/cxr/qxr_ln_data/LN_test/combined_test_csv_updated_internal_test_02-09-24.csv")

In [27]:
excel_file_path = '/fast_data_e2e_1/cxr/qxr_ln_data/FIT_lung_cancer/FIT Lung Cancer Pilot Data 2024-07-16.xlsx'
fit_df = pd.read_excel(excel_file_path)

# csv_file_path = '/fast_data_e2e_1/cxr/qxr_ln_data/FIT_lung_cancer/FIT_Lung_Cancer_Pilot_Data_2024-07-16.csv'
# df.to_csv(csv_file_path, index=False)

In [59]:
fit_df['nodule'] =1 
fit_df['LN'] =1 
fit_df['LNMS'] = fit_df['ct_result'] 
fit_df['LNMS'] = fit_df['LNMS'].map({'Malignant': 1, 'Benign': 0})
fit_df['src'] = 'FIT_lung_cancer'
fit_df.rename(columns={'acis_id':'filename'},inplace =True)
fit_df['png_path'] = "/fast_data_e2e_1/cxr/qxr_ln_data/FIT_lung_cancer/pngs/" + fit_df['filename'].astype(str) + '.png'
fit_df['bbox'] = [[] for _ in range(len(fit_df))]


In [67]:
fit_df_ = fit_df[['filename','src', 'nodule', 'LN' , 'LNMS', 'bbox', 'png_path']]

In [68]:
fit_df_

,filename,src,nodule,LN,LNMS,bbox,png_path
0,5449732,FIT_lung_cancer,1,1,1,[],/fast_data_e2e_1/cxr/qxr_ln_data/FIT_lung_canc...
1,8766760,FIT_lung_cancer,1,1,0,[],/fast_data_e2e_1/cxr/qxr_ln_data/FIT_lung_canc...
2,3627677,FIT_lung_cancer,1,1,0,[],/fast_data_e2e_1/cxr/qxr_ln_data/FIT_lung_canc...
3,8086712,FIT_lung_cancer,1,1,0,[],/fast_data_e2e_1/cxr/qxr_ln_data/FIT_lung_canc...
4,7357841,FIT_lung_cancer,1,1,0,[],/fast_data_e2e_1/cxr/qxr_ln_data/FIT_lung_canc...
...,...,...,...,...,...,...,...
495,4071719,FIT_lung_cancer,1,1,1,[],/fast_data_e2e_1/cxr/qxr_ln_data/FIT_lung_canc...
496,5802839,FIT_lung_cancer,1,1,1,[],/fast_data_e2e_1/cxr/qxr_ln_data/FIT_lung_canc...
497,5306358,FIT_lung_cancer,1,1,1,[],/fast_data_e2e_1/cxr/qxr_ln_data/FIT_lung_canc...
498,5958492,FIT_lung_cancer,1,1,1,[],/fast_data_e2e_1/cxr/qxr_ln_data/FIT_lung_canc...


In [69]:
tags_unknown = [ 'nipple_shadow', 'calcification',
       'granuloma', 'mass', 'ribfracture', 'consolidation', 'fibrosis',
       'ct recommended', 'solitary', 'artifact', 'subtlety', 'nipple_marker',
       'fibronodule', 'fibrolesion', 'miliarypattern', 'infectious_nodule',
       'tubr_nodule']
for tag in tags_unknown : 
    fit_df_[tag] = -100

/tmp/ipykernel_49768/3804143686.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fit_df_[tag] = -100
/tmp/ipykernel_49768/3804143686.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fit_df_[tag] = -100
/tmp/ipykernel_49768/3804143686.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning

In [71]:
all_testet = pd.concat([combined_test, fit_df_], axis=0, ignore_index=True)


In [131]:
all_testet

,filename,src,nodule,nipple_shadow,calcification,granuloma,mass,ribfracture,consolidation,fibrosis,...,nipple_marker,fibronodule,fibrolesion,miliarypattern,infectious_nodule,tubr_nodule,LN,LNMS,bbox,png_path
0,ca.50kstudies.unit6.a6b19ade890324758bc58aa5ad...,internal testset,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-100,[],/models_common_e2e/cxr_data/testing/images/ca....
1,naw.1.3.51.0.7.123293555.65006.58701.36014.869...,internal testset,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,-100,[],/models_common_e2e/cxr_data/testing/images/naw...
2,dmims.1.2.392.200036.9107.307.15120.20190123.1...,internal testset,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-100,[],/models_common_e2e/cxr_data/testing/images/dmi...
3,krsnaa.1.2.392.200036.9125.4.0.487578536.53798...,internal testset,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-100,[],/models_common_e2e/cxr_data/testing/images/krs...
4,krsnaa.1.2.392.200036.9125.9.0.253290553.41978...,internal testset,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,-100,[],/models_common_e2e/cxr_data/testing/images/krs...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
533845,4071719,FIT_lung_cancer,1,-100,-100,-100,-100,-100,-100,-100,...,-100,-100,-100,-100,-100,-100,1,1,[],/fast_data_e2e_1/cxr/qxr_ln_data/FIT_lung_canc...
533846,5802839,FIT_lung_cancer,1,-100,-100,-100,-100,-100,-100,-100,...,-100,-100,-100,-100,-100,-100,1,1,[],/fast_data_e2e_1/cxr/qxr_ln_data/FIT_lung_canc...
533847,5306358,FIT_lung_cancer,1,-100,-100,-100,-100,-100,-100,-100,...,-100,-100,-100,-100,-100,-100,1,1,[],/fast_data_e2e_1/cxr/qxr_ln_data/FIT_lung_canc...
533848,5958492,FIT_lung_cancer,1,-100,-100,-100,-100,-100,-100,-100,...,-100,-100,-100,-100,-100,-100,1,1,[],/fast_data_e2e_1/cxr/qxr_ln_data/FIT_lung_canc...


In [73]:
all_testet.to_csv("/fast_data_e2e_1/cxr/qxr_ln_data/LN_test/combined_test_csv_updated_internal_test_fit_cancer_02-09-24.csv", index = False)

### V4 metric update on cleaned testset 

In [93]:
v4_preds = pd.read_csv("/fast_data_e2e_1/cxr/qxr_ln_data/LN_test/v4_preds_combined_test_csv_25-7-24.csv")

In [94]:
v4_preds_fit = pd.read_csv("/fast_data_e2e_1/cxr/qxr_ln_data/LN_test/v4_preds_FIT_lung_cancer.csv")

In [95]:
v4_preds_comb = pd.concat([v4_preds, v4_preds_fit], axis=0, ignore_index=True)

In [96]:
v4_preds_comb

,regular_border,cancer,irregular_border,tinynodule,calcified,large,solitary,nodule,bboxes,png_path
0,0.139382,0.052841,0.016957,0.134351,0.011170,0.056637,0.229218,0.242255,[],/fast_data_e2e_1/cxr/qxr_ln_data/sbri/pngs/1.2...
1,0.350971,0.003724,0.018100,0.277812,0.036141,0.003958,0.359370,0.487087,"[(155, 570, 40, 43)]",/fast_data_e2e_1/cxr/qxr_ln_data/sbri/pngs/f26...
2,0.019424,0.000157,0.002667,0.018848,0.004317,0.000041,0.028076,0.058498,[],/fast_data_e2e_1/cxr/qxr_ln_data/sbri/pngs/02e...
3,0.234615,0.000722,0.005613,0.222408,0.021511,0.001725,0.247778,0.314716,[],/fast_data_e2e_1/cxr/qxr_ln_data/sbri/pngs/1c7...
4,0.004716,0.002701,0.000806,0.006558,0.006002,0.001042,0.008215,0.016818,[],/fast_data_e2e_1/cxr/qxr_ln_data/sbri/pngs/1.2...
...,...,...,...,...,...,...,...,...,...,...
536674,0.342339,0.118487,0.555795,0.010568,0.049245,0.926219,0.874426,0.935913,"[(262, 168, 78, 83)]",/fast_data_e2e_1/cxr/qxr_ln_data/FIT_lung_canc...
536675,0.223449,0.194996,0.608448,0.187537,0.095562,0.501588,0.549560,0.959161,"[(191, 227, 133, 109)]",/fast_data_e2e_1/cxr/qxr_ln_data/FIT_lung_canc...
536676,0.106341,0.264989,0.835446,0.012120,0.006038,0.941076,0.904604,0.990831,"[(288, 210, 73, 80)]",/fast_data_e2e_1/cxr/qxr_ln_data/FIT_lung_canc...
536677,0.379432,0.203656,0.494251,0.051491,0.014456,0.879309,0.873013,0.984903,"[(205, 260, 108, 143)]",/fast_data_e2e_1/cxr/qxr_ln_data/FIT_lung_canc...


In [114]:
for col in v4_preds_comb.columns: 
    if col != 'filename' and col != 'png_path': 
        v4_preds_comb.rename(columns={col: f'{col}_pred'}, inplace=True)

In [115]:
len(all_testet)

533850

In [119]:
all_testet_w_v4 = all_testet.merge(v4_preds_comb, on ='png_path' , how = "left")

In [120]:
all_testet_w_v4.dropna(inplace = True)

In [121]:
len(all_testet_w_v4)

533847

In [149]:
all_testet_w_v4.src.value_counts()

src
internal testset    314350
padchest            107709
segmed               66248
sbri                 31346
vrad                 11727
mgh                    676
nlst                   574
FIT_lung_cancer        500
fda                    470
jsrt                   247
Name: count, dtype: int64

In [132]:
all_testet_w_v4['lnms_pred']=  (all_testet_w_v4["solitary_pred"] + all_testet_w_v4["large_pred"] + all_testet_w_v4["cancer_pred"] + all_testet_w_v4["irregular_border_pred"])* (1 - all_testet_w_v4["tinynodule_pred"])* (1 - all_testet_w_v4["calcified_pred"])
all_testet_w_v4['lnms_pred'] = all_testet_w_v4['lnms_pred'] / 4

In [134]:
# all_testet_w_v4.to_csv("/fast_data_e2e_1/cxr/qxr_ln_data/LN_test/combined_test_csv_w_v4_preds_03-09-24.csv", index =False)

### calculate v4 metrics 

In [123]:
import numpy as np
from sklearn.metrics import (
    roc_curve, roc_auc_score, recall_score, precision_score, f1_score, accuracy_score, confusion_matrix
)

In [159]:
import numpy as np
from sklearn.metrics import (
    roc_curve, roc_auc_score, recall_score, precision_score, f1_score, accuracy_score, confusion_matrix
)

def get_metrics(df, target_col, pred_col):
    fpr, tpr, thresholds = roc_curve(df[target_col], df[pred_col])
    auc = roc_auc_score(df[target_col], df[pred_col])
    yoden_index = tpr - fpr
    best_threshold = thresholds[np.argmax(yoden_index)]
    yoden_index = np.max(yoden_index)
    sensitivity = recall_score(df[target_col], df[pred_col] > best_threshold)
    specificity = 1 - fpr[np.argmax(tpr > sensitivity)]
    precision = precision_score(df[target_col], df[pred_col] > best_threshold)
    recall = sensitivity
    f1 = f1_score(df[target_col], df[pred_col] > best_threshold)
    accuracy = accuracy_score(df[target_col], df[pred_col] > best_threshold)
    false_positives = confusion_matrix(df[target_col], df[pred_col] > best_threshold)[0][1]
    false_negatives = confusion_matrix(df[target_col], df[pred_col] > best_threshold)[1][0]
    return {
        'auc': auc,
        'yoden_index': yoden_index,
        'best_threshold': best_threshold,
        'sensitivity': sensitivity,
        'specificity': specificity,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'accuracy': accuracy,
        'false_positives': false_positives,
        'false_negatives': false_negatives
    }

# Specify column names
target_column_name = 'LN'
predicted_column_name = 'nodule_pred'

# Open a text file to log the results
with open("/fast_data_e2e_1/cxr/qxr_ln_data/LN_test/v4_test_metrics_log.txt", "w") as log_file:
    # Iterate over each unique src
    for src_name, group in all_testet_w_v4.groupby('src'):
        if src_name != "FIT_lung_cancer":
            # Filter out rows where target_col or pred_col have a value of -100
            filtered_group = group[(group[target_column_name] != -100) & (group[predicted_column_name] != -100)]
            
            # Skip if the filtered group is empty
            if filtered_group.empty:
                log_file.write(f"Skipping src: {src_name} (no valid data after filtering)\n")
                continue
            
            log_file.write(f"Metrics for src: {src_name}\n")
            metrics = get_metrics(filtered_group, target_col=target_column_name, pred_col=predicted_column_name)
            for key, value in metrics.items():
                log_file.write(f"{key}: {value}\n")
            log_file.write("\n" + "="*40 + "\n\n")
    
    # Calculate metrics for the entire DataFrame, after filtering
    filtered_all = all_testet_w_v4[(all_testet_w_v4[target_column_name] != -100) & 
                                   (all_testet_w_v4[predicted_column_name] != -100)]
    
    log_file.write("Metrics for overall data:\n")
    overall_metrics = get_metrics(filtered_all, target_col=target_column_name, pred_col=predicted_column_name)
    for key, value in overall_metrics.items():
        log_file.write(f"{key}: {value}\n")


In [160]:
all_testet_w_v4.src.value_counts()

src
internal testset    314350
padchest            107709
segmed               66248
sbri                 31346
vrad                 11727
mgh                    676
nlst                   574
FIT_lung_cancer        500
fda                    470
jsrt                   247
Name: count, dtype: int64

In [161]:
sources_to_log = ['jsrt', 'nlst', 'FIT_lung_cancer']

with open("/fast_data_e2e_1/cxr/qxr_ln_data/LN_test/v4_test_metrics_log.txt", "a") as log_file:
    log_file.write(f"\n\n")
    log_file.write("=" * 50 + "\n")
    for src in sources_to_log:
        # Filter the DataFrame for the current src
        filtered_df = all_testet_w_v4[all_testet['src'] == src]
        
        # Ensure there are no -100 values
        filtered_df = filtered_df[filtered_df['lnms_pred'] != -100]
        
        # Calculate AUC

        if not filtered_df.empty:
            auc = roc_auc_score(filtered_df['LNMS'], filtered_df['lnms_pred'])
            log_file.write(f"LNMS AUC for src {src}: {auc}\n")
        else:
            log_file.write(f"No valid data for src {src} after filtering.\n")


/tmp/ipykernel_49768/973377456.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_df = all_testet_w_v4[all_testet['src'] == src]
/tmp/ipykernel_49768/973377456.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_df = all_testet_w_v4[all_testet['src'] == src]
/tmp/ipykernel_49768/973377456.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_df = all_testet_w_v4[all_testet['src'] == src]


In [4]:
all_testet_w_v4 = pd.read_csv("/fast_data_e2e_1/cxr/qxr_ln_data/LN_test/combined_test_csv_w_v4_preds_03-09-24.csv")

In [ ]:
sources_to_log = ['jsrt', 'nlst', 'FIT_lung_cancer']

with open("/fast_data_e2e_1/cxr/qxr_ln_data/LN_test/v4_test_metrics_log.txt", "a") as log_file:
    log_file.write(f"\n\n")
    log_file.write("=" * 50 + "\n")
    for src in sources_to_log:
        # Filter the DataFrame for the current src
        filtered_df = all_testet_w_v4[all_testet['src'] == src]
        
        # Ensure there are no -100 values
        filtered_df = filtered_df[filtered_df['lnms_pred'] != -100]
        
        # Calculate AUC

        if not filtered_df.empty:
            auc = roc_auc_score(filtered_df['LNMS'], filtered_df['lnms_pred'])
            log_file.write(f"LNMS AUC for src {src}: {auc}\n")
        else:
            log_file.write(f"No valid data for src {src} after filtering.\n")


/tmp/ipykernel_49768/973377456.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_df = all_testet_w_v4[all_testet['src'] == src]
/tmp/ipykernel_49768/973377456.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_df = all_testet_w_v4[all_testet['src'] == src]
/tmp/ipykernel_49768/973377456.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_df = all_testet_w_v4[all_testet['src'] == src]


In [35]:
import os
import numpy as np
import cv2
import ast
from tqdm import tqdm  # Import tqdm for progress tracking

def calculate_combined_mask(bboxes_pred, img_shape):
    """Create a combined binary mask for all predicted bounding boxes."""
    
    combined_mask = np.zeros((960,960), dtype=np.uint8)
    # combined_mask = np.zeros(img_shape, dtype=np.uint8)

    for pred_bbox in bboxes_pred:
        x_min, y_min, w, h = pred_bbox
        x_max, y_max = x_min + w, y_min + h
        combined_mask[y_min:y_max, x_min:x_max] = 255  # Fill the bounding box area with white (255)

    return combined_mask

def load_mask(mask_path):
    """Load the mask from the given path. Assumes binary mask with values 0 or 1."""
    return cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

# Iterate over each source and its corresponding path
# sources_for_iou = ['jsrt', 'internal testset']
sources_for_iou = ['fda']
paths = [
    # '/fast_data_e2e_1/cxr/qxr_ln_data/jsrt_nlst/masks/nodule',
    # '/models_common_e2e/cxr_data/testing/test_annotations/nodule',
    '/fast_data_e2e_1/cxr/qxr_ln_data/fda_data/fda_masks/nodule'
    # '/fast_data_e2e_1/cxr/qxr_ln_data/fda_data/nodule_fda'
    
]

output_base_path = '/fast_data_e2e_1/cxr/qxr_ln_data/LN_test/v4_masks_preds'
# output_base_path = '/fast_data_e2e_1/cxr/qxr_ln_data/fda_data/fda_masks'

for source, path in zip(sources_for_iou, paths):
    print(f"Processing source: {source}")
    
    source_df = all_testet_w_v4[all_testet_w_v4['src'] == source]
    # Wrap the DataFrame iteration with tqdm for a progress bar
    for idx, row in tqdm(source_df.iterrows(), total=len(source_df), desc=f"Processing {source}"):
        bboxes_pred = row['bboxes_pred']  # List of lists: [[xmin, ymin, w, h], ...]
        filename = row['filename']
        
        # # Load the corresponding mask
        mask_path = os.path.join(path, filename + '.png')
        
        # Check if the mask file exists
        if not os.path.exists(mask_path):
            continue  # Skip this file if the mask doesn't exist

        mask = load_mask(mask_path)
        
        # Ensure the mask was loaded properly
        if mask is None:
            continue
        
        # if ast.literal_eval(bboxes_pred) == []: 
        #     print("empty bbox" )
        #     continue 
        
        # Generate a combined mask for all predicted bounding boxes
        combined_mask = calculate_combined_mask(ast.literal_eval(bboxes_pred), mask.shape)
        
        # Create the output folder if it doesn't exist
        output_folder = os.path.join(output_base_path, source)
        os.makedirs(output_folder, exist_ok=True)
        
        # Save the combined mask
        output_mask_path = os.path.join(output_folder, f'{filename}.png')
        cv2.imwrite(output_mask_path, combined_mask)
        
        # Optional: Logging can be done here if needed
        # print(f"Combined mask saved for {filename} in {output_folder}")


Processing source: fda


Processing fda:   0%|          | 0/470 [00:00<?, ?it/s]

Processing fda: 100%|██████████| 470/470 [00:12<00:00, 38.10it/s] 


In [29]:
source_df = all_testet_w_v4[all_testet_w_v4['src'] == 'fda']

In [32]:
len(source_df.bbox.value_counts())

180

In [36]:

def calculate_iou_between_masks(pred_mask, gt_mask, target_size=(960, 960)):
    """Calculate the IoU between the predicted mask and the ground truth mask."""
    
    # Resize the predicted mask to the target size (960, 960)
    resized_pred_mask = cv2.resize(pred_mask, target_size, interpolation=cv2.INTER_NEAREST)
    
    # Ensure the ground truth mask is also resized to match the target size if it's not already
    if gt_mask.shape != target_size:
        gt_mask = cv2.resize(gt_mask, target_size, interpolation=cv2.INTER_NEAREST)
    
    # Calculate intersection and union
    intersection = np.logical_and(resized_pred_mask, gt_mask)
    union = np.logical_or(resized_pred_mask, gt_mask)
    
    iou = np.sum(intersection) / np.sum(union)
    return iou

def compute_iou_for_sources(sources_for_iou, paths, output_base_path, gt_base_paths):
    """Calculate the IoU for each source by comparing predicted masks with ground truth masks."""
    iou_results = {}

    for source, pred_path, gt_path in zip(sources_for_iou, paths, gt_base_paths):
        print(f"Calculating IoU for source: {source}")
        
        # Initialize list to store IoU for each image
        iou_results[source] = []
        
        # Get the list of filenames in the predicted masks directory
        pred_files = os.listdir(os.path.join(output_base_path, source))
        
        for pred_file in tqdm(pred_files, desc=f"Processing {source}"):
            # Construct full paths to the predicted and ground truth masks
            pred_mask_path = os.path.join(output_base_path, source, pred_file)
            gt_mask_path = os.path.join(gt_path, pred_file)
            
            # Load the predicted and ground truth masks
            pred_mask = cv2.imread(pred_mask_path, cv2.IMREAD_GRAYSCALE)
            gt_mask = cv2.imread(gt_mask_path, cv2.IMREAD_GRAYSCALE)
            
            # Ensure both masks are loaded properly
            if pred_mask is None or gt_mask is None:
                print(f"Skipping {pred_file} due to missing masks.")
                continue
            
            # Calculate IoU between the predicted and ground truth masks
            iou = calculate_iou_between_masks(pred_mask, gt_mask)
            iou_results[source].append((pred_file, iou))
    
    return iou_results

# Define ground truth base paths corresponding to each source
pred_paths = [
    # '/fast_data_e2e_1/cxr/qxr_ln_data/LN_test/v4_masks_preds/jsrt',
    # '/fast_data_e2e_1/cxr/qxr_ln_data/LN_test/v4_masks_preds/internal_testset',
    '/fast_data_e2e_1/cxr/qxr_ln_data/LN_test/v4_masks_preds'
]

# Example usage
# sources_for_iou = ['jsrt', 'internal testset', 'FIT_lung_cancer']
sources_for_iou =['fda']
gt_base_paths = [
    # '/fast_data_e2e_1/cxr/qxr_ln_data/jsrt_nlst/masks/nodule',
    # '/models_common_e2e/cxr_data/testing/test_annotations/nodule',
    '/fast_data_e2e_1/cxr/qxr_ln_data/fda_data/fda_masks/nodule'
]
output_base_path = '/fast_data_e2e_1/cxr/qxr_ln_data/LN_test/v4_masks_preds'

# Calculate IoU for each source
iou_results = compute_iou_for_sources(sources_for_iou, pred_paths, output_base_path, gt_base_paths)

# # Print IoU results for analysis
# for source, results in iou_results.items():
#     print(f"\nIoU Results for source: {source}")
#     for filename, iou in results:
#         print(f"Filename: {filename}, IoU: {iou:.4f}")


Calculating IoU for source: fda


Processing fda:   0%|          | 0/179 [00:00<?, ?it/s]

Processing fda:  82%|████████▏ | 146/179 [00:10<00:04,  8.03it/s]

In [27]:
iou_results

{'fda': [('S_1795715278393443796.png', 0.06956165925016623),
  ('S_6220704828398298389.png', 0.0),
  ('S_6476158281088343129.png', 0.17469310670443816),
  ('S_2920783897834726893.png', 0.0),
  ('S_7659585142956331743.png', 0.23863636363636365),
  ('S_1566786527921963180.png', 0.0),
  ('S_4449915863623886423.png', 0.0),
  ('S_6357810758590080862.png', 0.0),
  ('S_9867292921957765920.png', 0.0),
  ('S_7610473968013498990.png', 0.0),
  ('S_4975265134713483824.png', 0.0),
  ('S_5355503267362800391.png', 0.0),
  ('S_6583764739514877316.png', 0.36036036036036034),
  ('S_7635576124332263981.png', 0.0),
  ('S_6315605746759128927.png', 0.0),
  ('S_7686092828972544222.png', 0.24025341130604289),
  ('S_1365412455253480175.png', 0.0),
  ('S_9859174170272416027.png', 0.0),
  ('S_2039421430740067169.png', 0.0),
  ('S_2880599432582293751.png', 0.21563011456628478),
  ('S_1843964902599214021.png', 0.2696690179055887),
  ('S_1827176984801448532.png', 0.0),
  ('S_1838892486895787761.png', 0.0),
  ('S_76

In [28]:
# Calculate and print the mean IoU for each source
for source, results in iou_results.items():
    print(f"\nIoU Results for source: {source}")
    
    if results:
        # Extract the IoU values from the results
        iou_values = [iou for filename, iou in results]
        
        # Calculate the mean IoU
        mean_iou = np.mean(iou_values)
        
        # Print the mean IoU
        print(f"Mean IoU for source '{source}': {mean_iou:.4f}")
        
        # # Optionally, print individual IoU results for each file
        # for filename, iou in results:
        #     print(f"Filename: {filename}, IoU: {iou:.4f}")
    else:
        print(f"No valid IoU calculations for source '{source}'.")



IoU Results for source: fda
Mean IoU for source 'fda': 0.1192


In [15]:
import cv2

In [18]:
img = cv2.imread("/fast_data_e2e_1/cxr/qxr_ln_data/LN_test/v4_masks_preds/FIT_lung_cancer/9406452.png",  cv2.IMREAD_GRAYSCALE)

In [19]:
img.shape

(960, 960)